# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../output_data/cities.csv")

#limit number of pulls so that I dont run out of funds on API
# cities_df = cities_df.head(20)
cities_df

,Unnamed: 0,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,morgan city,29.70,-91.21,53.01,76,90,5.82,US,1607274746
1,1,angoche,-16.23,39.91,80.83,75,91,8.37,MZ,1607274747
2,2,sao jose da coroa grande,-8.90,-35.15,81.32,66,66,11.79,BR,1607274747
3,3,bengkulu,-3.80,102.27,76.32,87,100,4.36,ID,1607274748
4,4,punta arenas,-53.15,-70.92,64.40,55,75,25.28,CL,1607274748
...,...,...,...,...,...,...,...,...,...,...
544,544,mingyue,43.11,128.92,19.67,85,5,12.59,CN,1607274832
545,545,bandarbeyla,9.49,50.81,76.23,75,29,12.59,SO,1607274832
546,546,borovskoy,53.80,64.15,7.27,94,60,7.49,KZ,1607274832
547,547,nelson bay,-32.72,152.15,69.01,77,100,7.11,AU,1607274832


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

#create variables to store location and humidity for Heatmap
locations = cities_df[['Lat','Lng']].astype(float)

humidity = cities_df['Humidity'].astype(float)


In [4]:
# Create a Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#create drop statements for <80, >70 Temp, 0 cloudiness and wind speed <10

#find temp greater than 70 but less than 80
vacation_cities = cities_df[cities_df['Max temp']>70]
vacation_cities = vacation_cities[vacation_cities['Max temp']<80]
#find cities with no clouds
vacation_cities = vacation_cities[vacation_cities['Cloudiness']==0]
#find cities with less than 10mph wind
vacation_cities = vacation_cities[vacation_cities['Wind Speed']<10]


vacation_cities

,Unnamed: 0,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,la plata,-34.92,-57.95,73.00,42,0,1.99,AR,1607274619
18,18,mar del plata,-38.00,-57.56,75.20,33,0,9.17,AR,1607274750
63,63,moron,-34.65,-58.62,77.00,30,0,4.00,AR,1607274700
176,176,abu dhabi,24.47,54.37,78.80,78,0,3.36,AE,1607274776
186,186,iranshahr,27.20,60.68,75.20,27,0,4.70,IR,1607274778
213,213,memari,23.20,88.12,71.60,73,0,3.36,IN,1607274782
223,223,hansot,21.58,72.80,76.60,34,0,7.58,IN,1607274783
257,257,ormara,25.21,64.64,76.95,70,0,2.33,PK,1607274788
284,284,thaton,16.92,97.37,72.90,75,0,2.33,MM,1607274497
299,299,porbandar,21.64,69.61,76.60,53,0,1.41,IN,1607274795


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = vacation_cities
#create an empty Hotel name column in new dataframe
hotel_df['Hotel Name'] = ""
hotel_df

,Unnamed: 0,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
7,7,la plata,-34.92,-57.95,73.00,42,0,1.99,AR,1607274619,
18,18,mar del plata,-38.00,-57.56,75.20,33,0,9.17,AR,1607274750,
63,63,moron,-34.65,-58.62,77.00,30,0,4.00,AR,1607274700,
176,176,abu dhabi,24.47,54.37,78.80,78,0,3.36,AE,1607274776,
186,186,iranshahr,27.20,60.68,75.20,27,0,4.70,IR,1607274778,
213,213,memari,23.20,88.12,71.60,73,0,3.36,IN,1607274782,
223,223,hansot,21.58,72.80,76.60,34,0,7.58,IN,1607274783,
257,257,ormara,25.21,64.64,76.95,70,0,2.33,PK,1607274788,
284,284,thaton,16.92,97.37,72.90,75,0,2.33,MM,1607274497,
299,299,porbandar,21.64,69.61,76.60,53,0,1.41,IN,1607274795,


In [7]:
#geocoordinates
# target_coordinates = hotel_df[['Lat','Lng']]
target_radius = 5000
target_type = "hotel"

#set params dictonary

params = {
         "radius": target_radius,
         "type": target_type,
         "key":g_key}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, location in hotel_df.iterrows():
    #get lat and long from database
    target_coordinates = f"{location['Lat']},{location['Lng']}"
    #append results to params
    params['location'] = target_coordinates
#     print(params)
    response = requests.get(base_url, params=params)

    places_data = response.json()
    hotel_df.loc[index, 'Hotel Name'] = places_data['results'][1]['name']
# Print the json (pretty printed)
# print(json.dumps(places_data, indent=4, sort_keys=True))
hotel_df

,Unnamed: 0,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
7,7,la plata,-34.92,-57.95,73.00,42,0,1.99,AR,1607274619,Hotel Corregidor
18,18,mar del plata,-38.00,-57.56,75.20,33,0,9.17,AR,1607274750,Gran Hotel Mar del Plata habitaciones
63,63,moron,-34.65,-58.62,77.00,30,0,4.00,AR,1607274700,Stadium Ituzaingó
176,176,abu dhabi,24.47,54.37,78.80,78,0,3.36,AE,1607274776,Beach Rotana
186,186,iranshahr,27.20,60.68,75.20,27,0,4.70,IR,1607274778,فیزیو تراپی صمیم
213,213,memari,23.20,88.12,71.60,73,0,3.36,IN,1607274782,Anita Super Market অনিতা সুপার মার্কেট
223,223,hansot,21.58,72.80,76.60,34,0,7.58,IN,1607274783,Essar Petrol Pump
257,257,ormara,25.21,64.64,76.95,70,0,2.33,PK,1607274788,New Maritime Seafood Ormara
284,284,thaton,16.92,97.37,72.90,75,0,2.33,MM,1607274497,Tain Pyar Guest House
299,299,porbandar,21.64,69.61,76.60,53,0,1.41,IN,1607274795,Lords Inn Porbandar


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
# Create bank symbol layer
hotel_layer = gmaps.marker_layer(
    locations, info_box_content=hotel_info)

# Display figure
fig = gmaps.figure(center=hotel_df[["Lat", "Lng"]].iloc[0], zoom_level = 1.6)

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))